### Importovanje biblioteka i podataka

In [84]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scikitplot as skplt
from scipy import stats
%matplotlib inline
import warnings

warnings.filterwarnings('ignore')
import pickle
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.base import TransformerMixin
from sklearn.base import BaseEstimator, clone

from sklearn import model_selection, preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import roc_auc_score

from sklearn.feature_extraction.text import CountVectorizer

from sklearn.linear_model import LogisticRegression

In [85]:
pd.set_option("display.max_columns", 30)

In [86]:
train_df = pd.read_csv('input/train.tsv', delimiter='\t')
train_df.head()

url  urlid  \
0  http://www.bloomberg.com/news/2010-12-23/ibm-p...   4042   
1  http://www.popsci.com/technology/article/2012-...   8471   
2  http://www.menshealth.com/health/flu-fighting-...   1164   
3  http://www.dumblittleman.com/2007/12/10-foolpr...   6684   
4  http://bleacherreport.com/articles/1205138-the...   9006   

                                         boilerplate alchemy_category  \
0  {"title":"IBM Sees Holographic Calls Air Breat...         business   
1  {"title":"The Fully Electronic Futuristic Star...       recreation   
2  {"title":"Fruits that Fight the Flu fruits tha...           health   
3  {"title":"10 Foolproof Tips for Better Sleep "...           health   
4  {"title":"The 50 Coolest Jerseys You Didn t Kn...           sports   

  alchemy_category_score  avglinksize  commonlinkratio_1  commonlinkratio_2  \
0               0.789131     2.055556           0.676471           0.205882   
1               0.574147     3.677966           0.508021           0.288770   
2               0.996526     2.382883           0.562016           0.321705   
3               0.801248     1.543103           0.400000           0.100000   
4               0.719157     2.676471           0.500000           0.222222   

   commonlinkratio_3  commonlinkratio_4  compression_ratio  embed_ratio  \
0           0.047059           0.023529           0.443783          0.0   
1           0.213904           0.144385           0.468649          0.0   
2           0.120155           0.042636           0.525448          0.0   
3           0.016667           0.000000           0.480725          0.0   
4           0.123457           0.043210           0.446143          0.0   

   framebased  frameTagRatio  hasDomainLink  html_ratio  image_ratio is_news  \
0           0       0.090774              0    0.245831     0.003883       1   
1           0       0.098707              0    0.203490     0.088652       1   
2           0       0.072448              0    0.226402     0.120536       1   
3           0       0.095861              0    0.265656     0.035343       1   
4           0       0.024908              0    0.228887     0.050473       1   

   lengthyLinkDomain  linkwordscore news_front_page  \
0                  1             24               0   
1                  1             40               0   
2                  1             55               0   
3                  0             24               0   
4                  1             14               0   

   non_markup_alphanum_characters  numberOfLinks  numwords_in_url  \
0                            5424            170                8   
1                            4973            187                9   
2                            2240            258               11   
3                            2737            120                5   
4                           12032            162               10   

   parametrizedLinkRatio  spelling_errors_ratio  label  
0               0.152941               0.079130      0  
1               0.181818               0.125448      1  
2               0.166667               0.057613      1  
3               0.041667               0.100858      1  
4               0.098765               0.082569      0

In [87]:
train_df.drop(['urlid', 'is_news', 'framebased'], axis=1, inplace=True)
#URL kolona je izbačena nešto kasnije, pre enkodiranja.

### Priprema podataka
#### Ekstrakcija domena

In [88]:
#!pip install tldextract

In [89]:
import tldextract

In [90]:
train_df['domain'] = train_df['url'].apply(
    lambda url: tldextract.extract(url).domain)
train_df['subdomain'] = train_df['url'].apply(
    lambda url: tldextract.extract(url).subdomain)
train_df['suffix'] = train_df['url'].apply(
    lambda url: tldextract.extract(url).suffix)

In [91]:
train_df.suffix[train_df.groupby('suffix')['suffix'].transform('count').lt(
    2)] = 'once'

In [92]:
train_df.subdomain[train_df.groupby('subdomain')['subdomain'].transform(
    'count').lt(2)] = 'once'

In [93]:
train_df.domain[train_df.groupby('domain')['domain'].transform('count').lt(
    2)] = 'once'

In [94]:
train_df.drop('url', axis=1, inplace=True)

#### Boilerplate kolona

In [95]:
train_df['boilerplate'] = train_df['boilerplate'].replace(to_replace=':null',
                                                          value=':""',
                                                          regex=True)

In [96]:
train_df.boilerplate = train_df.boilerplate.map(eval)

In [97]:
train_df.boilerplate[1].keys()

dict_keys(['title', 'body', 'url'])

In [98]:
train_df = pd.concat([
    train_df.drop('boilerplate', axis=1), train_df['boilerplate'].apply(
        pd.Series)
],
                     axis=1)

In [99]:
train_df.drop(['related', 'url'], axis=1, inplace=True)

#### Nedostajuće vrednosti kao np.nan

Nedostajuće vrednosti se pojavljuju u okviru znaka pitanja.

In [100]:
train_df.replace(to_replace='?', value=np.nan, inplace=True)

In [101]:
train_df.head()

alchemy_category alchemy_category_score  avglinksize  commonlinkratio_1  \
0         business               0.789131     2.055556           0.676471   
1       recreation               0.574147     3.677966           0.508021   
2           health               0.996526     2.382883           0.562016   
3           health               0.801248     1.543103           0.400000   
4           sports               0.719157     2.676471           0.500000   

   commonlinkratio_2  commonlinkratio_3  commonlinkratio_4  compression_ratio  \
0           0.205882           0.047059           0.023529           0.443783   
1           0.288770           0.213904           0.144385           0.468649   
2           0.321705           0.120155           0.042636           0.525448   
3           0.100000           0.016667           0.000000           0.480725   
4           0.222222           0.123457           0.043210           0.446143   

   embed_ratio  frameTagRatio  hasDomainLink  html_ratio  image_ratio  \
0          0.0       0.090774              0    0.245831     0.003883   
1          0.0       0.098707              0    0.203490     0.088652   
2          0.0       0.072448              0    0.226402     0.120536   
3          0.0       0.095861              0    0.265656     0.035343   
4          0.0       0.024908              0    0.228887     0.050473   

   lengthyLinkDomain  linkwordscore news_front_page  \
0                  1             24               0   
1                  1             40               0   
2                  1             55               0   
3                  0             24               0   
4                  1             14               0   

   non_markup_alphanum_characters  numberOfLinks  numwords_in_url  \
0                            5424            170                8   
1                            4973            187                9   
2                            2240            258               11   
3                            2737            120                5   
4                           12032            162               10   

   parametrizedLinkRatio  spelling_errors_ratio  label          domain  \
0               0.152941               0.079130      0            once   
1               0.181818               0.125448      1          popsci   
2               0.166667               0.057613      1      menshealth   
3               0.041667               0.100858      1   dumblittleman   
4               0.098765               0.082569      0  bleacherreport   

  subdomain suffix                                              title  \
0       www    com  IBM Sees Holographic Calls Air Breathing Batte...   
1       www    com  The Fully Electronic Futuristic Starting Gun T...   
2       www    com  Fruits that Fight the Flu fruits that fight th...   
3       www    com                10 Foolproof Tips for Better Sleep    
4              com  The 50 Coolest Jerseys You Didn t Know Existed...   

                                                body  
0  A sign stands outside the International Busine...  
1  And that can be carried on a plane without the...  
2  Apples The most popular source of antioxidants...  
3  There was a period in my life when I had a lot...  
4  Jersey sales is a curious business Whether you...

#### Ostale kolone- korekcija tipova

In [102]:
train_df['alchemy_category_score'] = train_df['alchemy_category_score'].astype(
    dtype='float')
train_df['news_front_page'] = train_df['news_front_page'].astype(dtype='float')

#### Podela podataka na X i y

In [103]:
X = train_df.loc[:, train_df.columns != 'label']
y = train_df['label']

In [104]:
categorical_cols = list(X.select_dtypes(include='object').columns)
numerical_cols = list(X.select_dtypes(exclude='object').columns)

#### alchemy_category kolona
Ređe kategorije postaju jedna

In [105]:
train_df.alchemy_category.value_counts()

recreation            1229
arts_entertainment     941
business               880
health                 506
sports                 380
culture_politics       343
computer_internet      296
science_technology     289
gaming                  76
religion                72
law_crime               31
unknown                  6
weather                  4
Name: alchemy_category, dtype: int64

In [106]:
X.alchemy_category[X.groupby('alchemy_category')['alchemy_category'].transform(
    'count').lt(100)] = 'RARE'

In [107]:
categorical_cols = list(X.select_dtypes(include='object').columns)
numerical_cols = list(X.select_dtypes(exclude='object').columns)

In [108]:
X.reset_index(inplace=True, drop=True)  #proveriti da li treba
y.reset_index(inplace=True, drop=True)

### Podela podataka na train i test

In [109]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.3,
                                                    random_state=42)

### Imputacija nedostajućih vrednosti

#### fit i transform- train

In [110]:
class CustomImputer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.missing_map = {}

    def fit(self, X, y=None):
        num_atts = X.select_dtypes(exclude='object').columns.tolist()

        for att in num_atts:
            self.missing_map.update({att: X[att].median()})

        cat_atts = X.select_dtypes(include='object').columns.tolist()

        for att in cat_atts:
            self.missing_map.update({att: X[att].value_counts().index[0]})

        return (self)

    def transform(self, X, y=None):

        for att, impute_value in self.missing_map.items():
            X[att] = X[att].fillna(impute_value)
        return (X)

In [111]:
custom_imputer = CustomImputer().fit(X_train)
X_train = custom_imputer.transform(X_train)

In [29]:
with open('output/custom_imputer.pcl', 'wb') as picklefile:
    pickle.dump(custom_imputer, picklefile)

#### transform- test

In [112]:
X_test = custom_imputer.transform(X_test)

In [113]:
X_test = custom_imputer.transform(X_test)  #test df ne sadrži target kolonu.

In [114]:
categorical_cols = list(X_train.select_dtypes(include='object').columns)
numerical_cols = list(X_train.select_dtypes(exclude='object').columns)

In [115]:
categorical_cols

['alchemy_category', 'domain', 'subdomain', 'suffix', 'title', 'body']

In [116]:
X_train.reset_index(inplace=True, drop=True)  #proveriti da li treba
y_train.reset_index(inplace=True, drop=True)

X_test.reset_index(inplace=True, drop=True)  #proveriti da li treba
y_test.reset_index(inplace=True, drop=True)

### Izbacivanje kolona  i odvajanje za kasniju obradu

In [117]:
titles_train = X_train.title
titles_test = X_test.title
bodies_train = X_train.body
bodies_test = X_test.body
X_train.drop(['body', 'title'], inplace=True, axis=1)
X_test.drop(['body', 'title'], inplace=True, axis=1)

### Enkodiranje kategoričkih varijabli

In [118]:
categorical_cols = list(X_train.select_dtypes(include='object').columns)
numerical_cols = list(X_train.select_dtypes(exclude='object').columns)

In [119]:
categorical_cols

['alchemy_category', 'domain', 'subdomain', 'suffix']

In [38]:
import pickle
with open('output/numerical_cols.pcl', 'wb') as picklefile:
    pickle.dump(numerical_cols, picklefile)
with open('output/categorical_cols.pcl', 'wb') as picklefile:
    pickle.dump(categorical_cols, picklefile)

#### Enkodiranje- train

In [120]:
X_train.shape

(5176, 24)

In [121]:
enc = OneHotEncoder(dtype=np.int8,
                    handle_unknown='ignore').fit(X_train[categorical_cols])

X_new = pd.DataFrame(enc.transform(X_train[categorical_cols]).toarray())
X_new.columns = enc.get_feature_names(categorical_cols)

X_new.head()

alchemy_category_RARE  alchemy_category_arts_entertainment  \
0                      0                                    0   
1                      0                                    1   
2                      0                                    1   
3                      0                                    0   
4                      0                                    0   

   alchemy_category_business  alchemy_category_computer_internet  \
0                          0                                   0   
1                          0                                   0   
2                          0                                   0   
3                          0                                   0   
4                          0                                   0   

   alchemy_category_culture_politics  alchemy_category_health  \
0                                  0                        0   
1                                  0                        0   
2                                  0                        0   
3                                  1                        0   
4                                  0                        0   

   alchemy_category_recreation  alchemy_category_science_technology  \
0                            1                                    0   
1                            0                                    0   
2                            0                                    0   
3                            0                                    0   
4                            0                                    0   

   alchemy_category_sports  domain_1000awesomethings  domain_101cookbooks  \
0                        0                         0                    0   
1                        0                         0                    0   
2                        0                         0                    0   
3                        0                         0                    0   
4                        1                         0                    0   

   domain_13gb  domain_17andbaking  domain_24-7humor  domain_2wired2tired  \
0            0                   0                 0                    0   
1            0                   0                 0                    0   
2            0                   0                 0                    0   
3            0                   0                 0                    0   
4            0                   0                 0                    0   

   ...  suffix_ly  suffix_me  suffix_name  suffix_net  suffix_net.au  \
0  ...          0          0            0           0              0   
1  ...          0          0            0           1              0   
2  ...          0          0            0           0              0   
3  ...          0          0            0           0              0   
4  ...          0          0            0           0              0   

   suffix_no  suffix_nu  suffix_once  suffix_org  suffix_org.uk  suffix_ro  \
0          0          0            0           0              0          0   
1          0          0            0           0              0          0   
2          0          0            0           0              0          0   
3          0          0            0           0              0          0   
4          0          0            0           0              0          0   

   suffix_ru  suffix_se  suffix_tv  suffix_us  
0          0          0          0          0  
1          0          0          0          0  
2          0          0          0          0  
3          0          0          0          0  
4          0          0          0          0  

[5 rows x 1002 columns]

In [122]:
X_train = pd.concat([X_train[numerical_cols], X_new], axis=1)

In [41]:
X_train.isna().sum().sum()

0

In [123]:
X_train.shape

(5176, 1022)

In [42]:
with open('output/enc.pcl', 'wb') as picklefile:
    pickle.dump(enc, picklefile)

#### Enkodiranje- test

In [124]:
X_new = pd.DataFrame(enc.transform(X_test[categorical_cols]).toarray())
X_new.columns = enc.get_feature_names(categorical_cols)

In [125]:
X_test = pd.concat([X_test[numerical_cols], X_new], axis=1)

In [126]:
X_train.reset_index(inplace=True, drop=True)  #proveriti da li treba
y_train.reset_index(inplace=True, drop=True)
#just in case
X_test.reset_index(inplace=True, drop=True)  #proveriti da li treba
y_test.reset_index(inplace=True, drop=True)

###  CountVectorizer- titles and bodies

In [46]:
#!pip install nltk
#nltk.download('stopwords') #odkomentarisati pri prvom izvršavanju

In [127]:
from nltk.corpus import stopwords

print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [128]:
print(stopwords.fileids())

['arabic', 'azerbaijani', 'bengali', 'danish', 'dutch', 'english', 'finnish', 'french', 'german', 'greek', 'hungarian', 'indonesian', 'italian', 'kazakh', 'nepali', 'norwegian', 'portuguese', 'romanian', 'russian', 'slovene', 'spanish', 'swedish', 'tajik', 'turkish']


Nema kineskih karaktera. Zato dopunjavamo naš rečnik i kineskim vezivnim rečima.

In [129]:
#!pip install spacy

In [130]:
from spacy.lang.zh.stop_words import STOP_WORDS

stopwords_chinese = STOP_WORDS

from spacy.lang.ja.stop_words import STOP_WORDS

stopwords_japanese = STOP_WORDS

from spacy.lang.ar.stop_words import STOP_WORDS

stopwords_arabic = STOP_WORDS

In [131]:
stop_words_all = set(stopwords_japanese) | set(stopwords_chinese) | set(
    stopwords_arabic) | set(stopwords.words())

#### CountVectorizer- train set - titles

In [132]:
X_train.shape

(5176, 1022)

In [133]:
cv2 = CountVectorizer(ngram_range=(1, 2),
                      lowercase=True,
                      stop_words=stop_words_all,
                      dtype=np.int8,
                      min_df=0.01)
cv2.fit(titles_train)
transformed_titles_train = cv2.transform(titles_train)
df_cv = pd.DataFrame(transformed_titles_train.todense(),
                     columns=cv2.get_feature_names())

In [52]:
with open('output/cv_title.pcl', 'wb') as picklefile:
    pickle.dump(cv2, picklefile)

In [134]:
X_train = pd.concat([X_train, df_cv], axis=1)

In [135]:
X_train.shape

(5176, 1072)

#### CountVectorizer- test set - titles

In [136]:
transformed_titles_test = cv2.transform(titles_test)
df_cv = pd.DataFrame(transformed_titles_test.todense(),
                     columns=cv2.get_feature_names())

In [137]:
X_test = pd.concat([X_test, df_cv], axis=1)

#### CountVectorizer- train set - bodies

In [138]:
cv2 = CountVectorizer(ngram_range=(1, 2),
                      stop_words=stop_words_all,
                      dtype=np.int8,
                      min_df=0.1)
cv2.fit(bodies_train)
transformed_bodies_train = cv2.transform(bodies_train)
df_cv = pd.DataFrame(transformed_bodies_train.todense(),
                     columns=cv2.get_feature_names())

In [139]:
X_train = pd.concat([X_train, df_cv], axis=1)

In [140]:
X_train.shape

(5176, 1240)

In [58]:
with open('output/cv_body.pcl', 'wb') as picklefile:
    pickle.dump(cv2, picklefile)

#### CountVectorizer- test set - bodies

In [59]:
transformed_bodies_test = cv2.transform(bodies_test)
df_cv = pd.DataFrame(transformed_bodies_test.todense(),
                     columns=cv2.get_feature_names())

In [60]:
X_test = pd.concat([X_test, df_cv], axis=1)

### Skaliranje podataka

#### Skaliranje- train

In [61]:
scaler = MinMaxScaler()
scaler.fit(X_train[numerical_cols])
X_train[numerical_cols] = scaler.transform(X_train[numerical_cols])

In [62]:
with open('output/scaler.pcl', 'wb') as picklefile:
    pickle.dump(scaler, picklefile)

#### Skaliranje- test

In [63]:
X_test[numerical_cols] = scaler.transform(X_test[numerical_cols])

### Izbacivanje outliera

In [64]:
z = np.abs(stats.zscore(X_train[numerical_cols]))
X_train['is_outlier'] = (np.any(z > 3, axis=1))

In [65]:
X_train['label'] = y_train
del X_train['is_outlier']

In [66]:
y_train = X_train['label']
del X_train['label']

In [67]:
X_train.reset_index(inplace=True, drop=True)
y_train.reset_index(inplace=True, drop=True)

In [68]:
X_train.shape, y_train.shape

((5176, 1240), (5176,))

In [69]:
X_train = X_train.loc[:, ~X_train.columns.duplicated()]
X_test = X_test.loc[:, ~X_test.columns.duplicated()]

In [70]:
X_train.to_csv('input/X_train.csv')
X_test.to_csv('input/X_test.csv')
y_train.to_csv('input/y_train.csv')
y_test.to_csv('input/y_test.csv')

### Kreiranje modela

Metrika je area under the ROC curve. 

In [71]:
model = LogisticRegression()

In [72]:
model.fit(X_train, y_train)
roc_auc_score(y_train, model.predict(X_train))

0.8302490755371658

In [73]:
roc_auc_score(y_test, model.predict(X_test))

0.7820256865287921

In [74]:
#body je bio više poboljšao model, ali oni zajedno rezultiraju sa dosta dobrim rezultatima, boljim nego oba pojedinačno.
# iako alchemy_category ne pravi neku preveliku razliku, odlučeno je se nova podela usvoji.
#krenuli smo od base modela koji je na testu ima auc 0.649, a uz malo truda doveli smo ga do 0.788

In [75]:
roc_auc_score(y_test, model.predict(X_test))

0.7820256865287921